In [1]:
! pip install vit-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 7.8 MB/s eta 0:00:00


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from vit_pytorch.simple_vit_3d import SimpleViT
from vit_pytorch.simple_vit_3d import posemb_sincos_3d
from einops import rearrange
from PIL import Image

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [3]:
vit_3d = SimpleViT(
    image_size = 128,          # image size
    frames = 16,               # number of frames
    image_patch_size = 16,     # image patch size
    frame_patch_size = 2,      # frame patch size
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048
)

#example input:
#video = torch.randn(4, 3, 16, 128, 128) # (batch, channels, frames, height, width)

#embedding size will be (4, 1000)
#preds = vit_3d(video) # (4, 1000)

In [4]:
#source code for these conv block implemenations is from this UNETR implementation: https://github.com/tamasino52/UNETR/blob/main/unetr.py#L231
class SingleDeconv3DBlock(nn.Module):
    def __init__(self, in_planes, out_planes):
        super().__init__()
        self.block = nn.ConvTranspose3d(in_planes, out_planes, kernel_size=2, stride=2, padding=0, output_padding=0)

    def forward(self, x):
        return self.block(x)


class SingleConv3DBlock(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size):
        super().__init__()
        self.block = nn.Conv3d(in_planes, out_planes, kernel_size=kernel_size, stride=1,
                               padding=((kernel_size - 1) // 2))

    def forward(self, x):
        return self.block(x)


class Conv3DBlock(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size=3):
        super().__init__()
        self.block = nn.Sequential(
            SingleConv3DBlock(in_planes, out_planes, kernel_size),
            nn.BatchNorm3d(out_planes),
            nn.ReLU(True)
        )

    def forward(self, x):
        return self.block(x)


class Deconv3DBlock(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size=3):
        super().__init__()
        self.block = nn.Sequential(
            SingleDeconv3DBlock(in_planes, out_planes),
            SingleConv3DBlock(out_planes, out_planes, kernel_size),
            nn.BatchNorm3d(out_planes),
            nn.ReLU(True)
        )

    def forward(self, x):
        return self.block(x)

In [9]:
class SimpleViT3dSeg(SimpleViT):
  #add decoder attributes for segmentation
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

    print(kwargs['dim'], kwargs['num_classes'])
    #upsampling decoder, from patch size 8x8x8 to 128x128x128
    self.decoder = nn.Sequential(
      Deconv3DBlock(kwargs['dim'], 256),
      Deconv3DBlock(256, 128),
      Deconv3DBlock(128, 64),
      nn.Conv3d(in_channels=64, out_channels=kwargs['num_classes'], kernel_size=1)
    )

  #override the forward function so embeddings get fed into decoder instead of linear classification head
  def forward(self, video):
        *_, h, w, dtype = *video.shape, video.dtype

        x = self.to_patch_embedding(video)
        #print(f"patch shape: {x.shape}")
        _, depth_patch_size, height_patch_size, width_patch_size, _ = x.shape

        pe = posemb_sincos_3d(x)
        x = rearrange(x, 'b ... d -> b (...) d') + pe

        x = self.transformer(x)
        #print(f"raw embeddings shape: {x.shape}")
        batch_size, patch_volume, embd_size = x.shape

        #convert to per-patch embedding format for segmentation
        #credit: code for re-arranging to per-patch format generated from chatgpt
        feat_grid_embeddings = x.transpose(1, 2).contiguous().view(batch_size, embd_size, depth_patch_size, height_patch_size, width_patch_size)
        #print(f"per-patch embedding dim: {feat_grid_embeddings.shape}")

        #x = x.mean(dim = 1) #don't apply pooling since we want the per-patch embedding
        feat_grid_embeddings = self.to_latent(feat_grid_embeddings) #this is a palce holder, does nothing so we can keep it

        logits = self.decoder(feat_grid_embeddings)
        return logits

In [13]:
test_vit_seg_model = SimpleViT3dSeg(
    image_size = 64,          # image size
    frames = 64,               # for volumetric data: this is slice number/depth
    image_patch_size = 8,     # image patch size
    frame_patch_size = 8,      # for volumetric data: this should be same as image patch size
    num_classes = 3,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048,
)

test_input = torch.randn(1, 3, 64, 64, 64)

preds = test_vit_seg_model(test_input)

print(preds.shape, type(preds))

1024 3
torch.Size([1, 3, 64, 64, 64]) <class 'torch.Tensor'>


In [20]:
#chatgpt generated using prompt "generate a simple function that generates a 3 channel white cube, inside it is a red sphere of varied position and radius"

def make_rgb_cube_with_red_sphere(T=64, H=64, W=64, cube_margin=8,
                                  min_radius=4, device="cpu"):
    """
    Returns:
      img:  (3, T, H, W) float32 in [0,1]
      mask: (T, H, W)    long: 0=bg, 1=cube, 2=sphere
    """
    # --- init ---
    img  = torch.zeros(3, T, H, W, dtype=torch.float32, device=device)  # RGB
    mask = torch.zeros(T, H, W, dtype=torch.long, device=device)        # seg labels

    # --- define cube bounds (axis-aligned) ---
    z0, z1 = cube_margin, T - cube_margin
    y0, y1 = cube_margin, H - cube_margin
    x0, x1 = cube_margin, W - cube_margin

    # white cube
    img[:, z0:z1, y0:y1, x0:x1] = 1.0
    mask[  z0:z1, y0:y1, x0:x1] = 1

    # --- random sphere INSIDE the cube ---
    # max radius so sphere fits fully inside cube
    max_r = min((z1 - z0), (y1 - y0), (x1 - x0)) // 3
    r = int(torch.randint(min_radius, max( min_radius+1, max_r+1 ), ()).item())

    cz = int(torch.randint(z0 + r, z1 - r, ()).item())
    cy = int(torch.randint(y0 + r, y1 - r, ()).item())
    cx = int(torch.randint(x0 + r, x1 - r, ()).item())

    # build sphere mask
    zz = torch.arange(T, device=device).view(T, 1, 1)
    yy = torch.arange(H, device=device).view(1, H, 1)
    xx = torch.arange(W, device=device).view(1, 1, W)
    sphere = (zz - cz)**2 + (yy - cy)**2 + (xx - cx)**2 <= r**2

    # paint sphere: red on top of cube
    img[0][sphere] = 1.0   # R
    img[1][sphere] = 0.0   # G
    img[2][sphere] = 0.0   # B
    mask[sphere]    = 2

    return img.contiguous(), mask.contiguous()

#chatgpt generated using prompt "generate a function to visualize the generated data"
def save_gif(img, path="volume.gif"):
    _, T, _, _ = img.shape
    frames = []
    for i in range(T):
        slice_rgb = (img[:, i].permute(1,2,0).cpu().numpy() * 255).astype("uint8")
        frames.append(Image.fromarray(slice_rgb))
    frames[0].save(path, save_all=True, append_images=frames[1:], duration=100, loop=0)

In [26]:
#generate dataset
train_img = []
train_mask = []
test_img = []
test_mask = []
for i in range(100):
  img, mask = make_rgb_cube_with_red_sphere(device="cuda")
  train_img.append(img)
  train_mask.append(mask)

for i in range(30):
  img, mask = make_rgb_cube_with_red_sphere(device="cuda")
  test_img.append(img)
  test_mask.append(mask)

train_loader = DataLoader(TensorDataset(torch.stack(train_img), torch.stack(train_mask)), batch_size=8, shuffle=True)
test_loader = DataLoader(TensorDataset(torch.stack(test_img), torch.stack(test_mask)), batch_size=8, shuffle=True)

In [25]:
img, mask = make_rgb_cube_with_red_sphere(device="cuda")
save_gif(img, "volume.gif")

In [ ]:
#training loop
vit_3d_seg = SimpleViT3dSeg(
    image_size = 64,
    frames = 64,
    image_patch_size = 8,
    frame_patch_size = 8,
    num_classes = 3,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 100
ce_loss = nn.CrossEntropyLoss()
vit_3d_seg.to(device)
optimizer = optim.Adam(vit_3d_seg.parameters(), lr=1e-3, weight_decay=1e-2)

for ep in range(epochs):
  vit_3d_seg.train()
  train_loss = 0.0

  for x, y in train_loader:
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad()
    logits = vit_3d_seg(x)
    loss = ce_loss(logits, y)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  train_loss = train_loss / len(train_loader)

  #eval
  vit_3d_seg.eval()
  test_loss = 0.0
  with torch.no_grad():
    for x, y in test_loader:
      x, y = x.to(device), y.to(device)
      logits = vit_3d_seg(x)
      loss = ce_loss(logits, y)
      test_loss += loss.item()
    test_loss = test_loss / len(test_loader)

  print(f"epoch: {ep}, train loss: {train_loss:.4f}, test loss: {test_loss:.4f}")


1024 3
epoch: 0, train loss: 0.6391, test loss: 12.1639
epoch: 1, train loss: 0.3359, test loss: 1.2868
epoch: 2, train loss: 0.2821, test loss: 1.8189
epoch: 3, train loss: 0.2135, test loss: 2.2810
epoch: 4, train loss: 0.1847, test loss: 0.4686
epoch: 5, train loss: 0.1167, test loss: 1.4949
